In [1]:
import numpy as np
import pandas as pd
import sqlite3

In [2]:
data_url = 'https://raw.githubusercontent.com/jkropko/contrans/main/examples/ASA%20All%20NBA%20Raw%20Data.csv'
nba = pd.read_csv(data_url)

In [3]:
nba.columns

Index(['game_id', 'game_date', 'OT', 'H_A', 'Team_Abbrev', 'Team_Score',
       'Team_pace', 'Team_efg_pct', 'Team_tov_pct', 'Team_orb_pct',
       'Team_ft_rate', 'Team_off_rtg', 'Inactives', 'Opponent_Abbrev',
       'Opponent_Score', 'Opponent_pace', 'Opponent_efg_pct',
       'Opponent_tov_pct', 'Opponent_orb_pct', 'Opponent_ft_rate',
       'Opponent_off_rtg', 'player', 'player_id', 'starter', 'mp', 'fg', 'fga',
       'fg_pct', 'fg3', 'fg3a', 'fg3_pct', 'ft', 'fta', 'ft_pct', 'orb', 'drb',
       'trb', 'ast', 'stl', 'blk', 'tov', 'pf', 'pts', 'plus_minus',
       'did_not_play', 'is_inactive', 'ts_pct', 'efg_pct', 'fg3a_per_fga_pct',
       'fta_per_fga_pct', 'orb_pct', 'drb_pct', 'trb_pct', 'ast_pct',
       'stl_pct', 'blk_pct', 'tov_pct', 'usg_pct', 'off_rtg', 'def_rtg', 'bpm',
       'season', 'minutes', 'double_double', 'triple_double', 'DKP', 'FDP',
       'SDP', 'DKP_per_minute', 'FDP_per_minute', 'SDP_per_minute',
       'pf_per_minute', 'ts', 'last_60_minutes_per_game_s

In [4]:
nba = nba[['game_id', 'game_date', 'OT', 'H_A', 'Team_Abbrev', 'Team_Score',
       'Team_pace', 'Team_efg_pct', 'Team_tov_pct', 'Team_orb_pct',
       'Team_ft_rate', 'Team_off_rtg', 'Inactives', 'Opponent_Abbrev',
       'player', 'player_id', 'starter', 'mp', 'fg', 'fga',
       'fg_pct', 'fg3', 'fg3a', 'fg3_pct', 'ft', 'fta', 'ft_pct', 'orb', 'drb',
       'trb', 'ast', 'stl', 'blk', 'tov', 'pf', 'pts', 'plus_minus',
       'did_not_play', 'is_inactive', 'off_rtg', 'def_rtg', 'bpm']]

In [5]:
pd.set_option('display.max_rows', 42)
pd.set_option('display.max_columns', 42)
nba.head(3)

,game_id,game_date,OT,H_A,Team_Abbrev,Team_Score,Team_pace,Team_efg_pct,Team_tov_pct,Team_orb_pct,Team_ft_rate,Team_off_rtg,Inactives,Opponent_Abbrev,player,player_id,starter,mp,fg,fga,fg_pct,fg3,fg3a,fg3_pct,ft,fta,ft_pct,orb,drb,trb,ast,stl,blk,tov,pf,pts,plus_minus,did_not_play,is_inactive,off_rtg,def_rtg,bpm
0,202204100BRK,2022-04-10,0,A,IND,126,103.9,0.543,5.9,20.8,0.125,121.3,"Chris Duarte, Nate Hinton, Ricky Rubio, Myles ...",BRK,Tyrese Haliburton,halibty01,1,39:28,7,14,0.500,2,5,0.400,1,1,1.00,2,2,4,10,2,0,1,0,17,-9,0,0,137,132,1.7
1,202204100BRK,2022-04-10,0,A,IND,126,103.9,0.543,5.9,20.8,0.125,121.3,"Chris Duarte, Nate Hinton, Ricky Rubio, Myles ...",BRK,Buddy Hield,hieldbu01,1,35:53,8,23,0.348,5,14,0.357,0,0,0.00,0,3,3,6,3,0,2,3,21,0,0,0,94,128,-2.3
2,202204100BRK,2022-04-10,0,A,IND,126,103.9,0.543,5.9,20.8,0.125,121.3,"Chris Duarte, Nate Hinton, Ricky Rubio, Myles ...",BRK,Oshae Brissett,brissos01,1,35:47,10,20,0.500,5,10,0.500,3,4,0.75,3,5,8,3,1,0,0,5,28,-9,0,0,137,133,4.4


## First normal form

- Every table has a primary key? Yes.

- Non-atomic data?

In [6]:
nba = nba.drop(['Inactives'], axis=1)
nba.head(3).T

,0,1,2
game_id,202204100BRK,202204100BRK,202204100BRK
game_date,2022-04-10,2022-04-10,2022-04-10
OT,0,0,0
H_A,A,A,A
Team_Abbrev,IND,IND,IND
Team_Score,126,126,126
Team_pace,103.9,103.9,103.9
Team_efg_pct,0.543,0.543,0.543
Team_tov_pct,5.9,5.9,5.9
Team_orb_pct,20.8,20.8,20.8


 - Repeating groups? No.
 
 - First normal form!

## Second normal form

- Data in first normal form? Yes. 

- Every non-prime column must depend on the ENTIRE primary key (gameid + playerid) and not just part of the primary key

In [7]:
nba['game_player_id'] = nba['game_id'] + '_' + nba['player_id']

In [8]:
nba.head(3).T

,0,1,2
game_id,202204100BRK,202204100BRK,202204100BRK
game_date,2022-04-10,2022-04-10,2022-04-10
OT,0,0,0
H_A,A,A,A
Team_Abbrev,IND,IND,IND
Team_Score,126,126,126
Team_pace,103.9,103.9,103.9
Team_efg_pct,0.543,0.543,0.543
Team_tov_pct,5.9,5.9,5.9
Team_orb_pct,20.8,20.8,20.8


## Third normal form

- Data must be in second normal form

- Calculated column:

fg_pct

fg3_pct

ft_pct

In [13]:
nba = nba.drop(['fg_pct', 'fg3_pct', 'ft_pct', 'trb'], axis=1)

In [14]:
nba.head(3).T

,0,1,2
game_id,202204100BRK,202204100BRK,202204100BRK
game_date,2022-04-10,2022-04-10,2022-04-10
OT,0,0,0
H_A,A,A,A
Team_Abbrev,IND,IND,IND
Team_Score,126,126,126
Team_pace,103.9,103.9,103.9
Team_efg_pct,0.543,0.543,0.543
Team_tov_pct,5.9,5.9,5.9
Team_orb_pct,20.8,20.8,20.8


- Transitive dependencies:

Some columns depend on player

Some columns depend on game

some columns depend on team

some columns depend on team + game

Each of these will get a separate table

In [15]:
nba.columns

Index(['game_id', 'game_date', 'OT', 'H_A', 'Team_Abbrev', 'Team_Score',
       'Team_pace', 'Team_efg_pct', 'Team_tov_pct', 'Team_orb_pct',
       'Team_ft_rate', 'Team_off_rtg', 'Opponent_Abbrev', 'player',
       'player_id', 'starter', 'mp', 'fg', 'fga', 'fg3', 'fg3a', 'ft', 'fta',
       'orb', 'drb', 'ast', 'stl', 'blk', 'tov', 'pf', 'pts', 'plus_minus',
       'did_not_play', 'is_inactive', 'off_rtg', 'def_rtg', 'bpm',
       'game_player_id'],
      dtype='object')

In [19]:
nba_teamgame = nba[['game_id', 'Team_Abbrev', 'H_A', 'Opponent_Abbrev', 'Team_Score',
       'Team_pace', 'Team_efg_pct', 'Team_tov_pct', 'Team_orb_pct',
       'Team_ft_rate', 'Team_off_rtg']]
nba_teamgame = nba_teamgame.drop_duplicates()
nba_teamgame

,game_id,Team_Abbrev,H_A,Opponent_Abbrev,Team_Score,Team_pace,Team_efg_pct,Team_tov_pct,Team_orb_pct,Team_ft_rate,Team_off_rtg
0,202204100BRK,IND,A,BRK,126,103.9,0.543,5.9,20.8,0.125,121.3
12,202204100BRK,BRK,H,IND,134,103.9,0.691,17.9,29.6,0.272,129.0
25,202204100CHO,WAS,A,CHO,108,97.7,0.489,8.7,31.5,0.170,110.5
37,202204100CHO,CHO,H,WAS,124,97.7,0.640,15.2,29.7,0.112,126.9
52,202204100CLE,MIL,A,CLE,115,101.9,0.511,10.5,17.4,0.284,112.9
...,...,...,...,...,...,...,...,...,...,...,...
21491,202112190MIN,MIN,H,DAL,111,91.8,0.565,9.0,15.0,0.312,120.9
21492,202112210DAL,MIN,A,DAL,102,93.9,0.538,16.2,20.9,0.215,108.6
21629,202112280MIN,MIN,H,NYK,88,93.1,0.441,9.6,20.4,0.153,94.5
21708,202112230UTA,MIN,A,UTA,116,102.1,0.530,11.6,30.0,0.089,113.7


In [20]:
nba_game = nba[['game_id', 'game_date', 'OT']].drop_duplicates()
nba_game

,game_id,game_date,OT
0,202204100BRK,2022-04-10,0
25,202204100CHO,2022-04-10,0
52,202204100CLE,2022-04-10,0
77,202204100DAL,2022-04-10,0
103,202204100DEN,2022-04-10,1
...,...,...,...
19708,202110300MIN,2021-10-30,0
19726,202112150DEN,2021-12-15,0
19748,202202010MIN,2022-02-01,0
20615,202203270BOS,2022-03-27,0


In [23]:
nba_players = nba[['player_id', 'player']].drop_duplicates()
nba_players

,player_id,player
0,halibty01,Tyrese Haliburton
1,hieldbu01,Buddy Hield
2,brissos01,Oshae Brissett
3,jacksis01,Isaiah Jackson
4,mccontj01,T.J. McConnell
...,...,...
31515,garrema01,Marcus Garrett
31535,chalmma01,Mario Chalmers
31538,holmaar01,Aric Holman
31540,scrubja01,Jay Scrubb


In [25]:
nba_playergame = nba.drop(['player', 'game_date', 'OT', 'H_A', 'Team_Abbrev', 'Opponent_Abbrev', 'Team_Score',
       'Team_pace', 'Team_efg_pct', 'Team_tov_pct', 'Team_orb_pct',
       'Team_ft_rate', 'Team_off_rtg'], axis=1)
nba_playergame

,game_id,player_id,starter,mp,fg,fga,fg3,fg3a,ft,fta,orb,drb,ast,stl,blk,tov,pf,pts,plus_minus,did_not_play,is_inactive,off_rtg,def_rtg,bpm,game_player_id
0,202204100BRK,halibty01,1,39:28,7,14,2,5,1,1,2,2,10,2,0,1,0,17,-9,0,0,137,132,1.7,202204100BRK_halibty01
1,202204100BRK,hieldbu01,1,35:53,8,23,5,14,0,0,0,3,6,3,0,2,3,21,0,0,0,94,128,-2.3,202204100BRK_hieldbu01
2,202204100BRK,brissos01,1,35:47,10,20,5,10,3,4,3,5,3,1,0,0,5,28,-9,0,0,137,133,4.4,202204100BRK_brissos01
3,202204100BRK,jacksis01,1,32:01,3,4,0,0,1,2,0,3,0,2,1,2,5,7,3,0,0,89,128,-9.2,202204100BRK_jacksis01
4,202204100BRK,mccontj01,1,30:52,5,15,3,7,1,2,0,3,5,3,0,0,3,14,7,0,0,104,126,-1.7,202204100BRK_mccontj01
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
31603,202201130NOP,gabriwe01,0,4:26,1,1,1,1,0,0,1,2,0,0,0,2,2,3,-4,0,0,62,110,-6.4,202201130NOP_gabriwe01
31604,202201150SAS,gabriwe01,0,0:00,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0.0,202201150SAS_gabriwe01
31605,202112220SAC,wrighmo01,0,1:28,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,217,103,24.4,202112220SAC_wrighmo01
31606,202112260LAC,wrighmo01,0,0:00,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0.0,202112260LAC_wrighmo01


In [27]:
nba_db = sqlite3.connect('nba.db')

In [28]:
nba_game.to_sql('games', nba_db, index=False, chunksize = 1000, if_exists = 'replace')
nba_playergame.to_sql('player_game', nba_db, index=False, chunksize = 1000, if_exists = 'replace')
nba_teamgame.to_sql('team_game', nba_db, index=False, chunksize = 1000, if_exists = 'replace')
nba_players.to_sql('players', nba_db, index=False, chunksize = 1000, if_exists = 'replace')

621

In [29]:
myquery = '''
SELECT * 
FROM team_game
WHERE Team_Abbrev='CLE'
'''

pd.read_sql(myquery, nba_db)

,game_id,Team_Abbrev,H_A,Opponent_Abbrev,Team_Score,Team_pace,Team_efg_pct,Team_tov_pct,Team_orb_pct,Team_ft_rate,Team_off_rtg
0,202204100CLE,CLE,H,MIL,133,101.9,0.644,9.0,23.3,0.128,130.5
1,202204080BRK,CLE,A,BRK,107,89.6,0.537,8.0,23.8,0.232,119.4
2,202204050ORL,CLE,A,ORL,115,96.8,0.581,10.3,20.5,0.174,118.8
3,202204030CLE,CLE,H,PHI,108,95.0,0.545,8.9,15.6,0.295,113.7
4,202204020NYK,CLE,A,NYK,119,92.1,0.636,9.1,19.4,0.198,129.2
...,...,...,...,...,...,...,...,...,...,...,...
77,202112300WAS,CLE,A,WAS,93,94.2,0.470,11.4,17.8,0.181,98.7
78,202112310CLE,CLE,H,ATL,118,93.2,0.619,8.7,21.1,0.167,126.7
79,202201020CLE,CLE,H,IND,108,92.8,0.500,7.4,28.6,0.227,116.4
80,202201310CLE,CLE,H,NOP,93,88.5,0.482,9.1,16.7,0.143,105.0


In [32]:
nba_game.columns

Index(['game_id', 'game_date', 'OT'], dtype='object')

In [33]:
nba_players.columns

Index(['player_id', 'player'], dtype='object')

In [34]:
nba_teamgame.columns

Index(['game_id', 'Team_Abbrev', 'H_A', 'Opponent_Abbrev', 'Team_Score',
       'Team_pace', 'Team_efg_pct', 'Team_tov_pct', 'Team_orb_pct',
       'Team_ft_rate', 'Team_off_rtg'],
      dtype='object')

In [35]:
nba_playergame.columns

Index(['game_id', 'player_id', 'starter', 'mp', 'fg', 'fga', 'fg3', 'fg3a',
       'ft', 'fta', 'orb', 'drb', 'ast', 'stl', 'blk', 'tov', 'pf', 'pts',
       'plus_minus', 'did_not_play', 'is_inactive', 'off_rtg', 'def_rtg',
       'bpm', 'game_player_id'],
      dtype='object')